# core 

> modifying streamlit methods to play nice with jupyter

Currently streamlit expects to find itself in a standalone python script, and will not work in a jupyter notebook. This is because streamlit is a web app, and jupyter notebooks are not.

To get around this, we can use the `get_ipython` method to check if we are in a jupyter notebook, and if so, we can use the `display` method to display the streamlit app in the notebook.

This allows us to interactively code in our jupyter notebook and see how our streamlit app will look in real time.

In [ ]:
# | default_exp core

In [ ]:
# | export

import functools
import logging
import time
import typing as tp
from datetime import datetime

import IPython.display
import ipywidgets as widgets
import pandas as pd
import streamlit as st
from fastcore.basics import in_ipython, listify, noop, patch, patch_to
from fastcore.test import test_eq, test_fail
from IPython.utils.capture import capture_output
from nbdev.showdoc import show_doc

In [ ]:
# | exporti
from logging import getLogger

logger = getLogger(__name__)

In [ ]:
# | export

# module obljects that we will be importing
IN_IPYTHON = in_ipython()

In [ ]:
assert IN_IPYTHON, "This module is intended to be used in a Jupyter notebook"

## Calling tqdm as tqdm.notebook or stqdm depending on environment

In [ ]:
# | export
if IN_IPYTHON:
    from tqdm.notebook import tqdm
else:
    from stqdm import stqdm as tqdm

tqdm = tqdm  # make this available in the module namespace

In [ ]:
# | export


class StreamlitPatcher:
    """class to patch streamlit functions for displaying content in jupyter notebooks"""

    def __init__(self):
        self.is_registered: bool = False
        self.registered_methods: tp.Set[str] = set()

    def jupyter(self, verbose: bool = False):
        """patches streamlit methods to display content in jupyter notebooks"""
        # patch streamlit methods from MAPPING property dict
        for method_name, wrapper in self.MAPPING.items():
            self._wrap(method_name, wrapper, verbose=verbose)

        self.is_registered = True

    @staticmethod
    def _get_streamlit_methods():
        """get all streamlit methods"""
        return [attr for attr in dir(st) if not attr.startswith("_")]

## _wrap method to patch streamlit methods

In [ ]:
# | exporti
@patch_to(StreamlitPatcher, cls_method=False)
def _wrap(
    cls,
    method_name: str,
    wrapper: tp.Callable,
    verbose: bool = False,
) -> None:
    """make a streamlit method jupyter friendly

    Parameters
    ----------
    method_name : str
        which method to jupyterify
    wrapper : tp.Callable
        wrapper function to use
    """
    if IN_IPYTHON:  # only patch if in jupyter
        if verbose:
            if hasattr(wrapper, "__name__"):
                print(
                    f"wrapping 'st.{method_name}' with 'streamlit_jupyter.core.{wrapper.__name__}'"
                )
            else:
                print(
                    f"wrapping 'st.{method_name}' with 'streamlit_jupyter.core.{wrapper}'"
                )

        trg = getattr(st, method_name)  # get the streamlit method
        setattr(st, method_name, wrapper(trg))  # patch the method
        cls.registered_methods.add(method_name)  # add to registered methods

In [ ]:
sp = StreamlitPatcher()

assert not sp.is_registered, "StreamlitPatcher is already registered"

In [ ]:
show_doc(StreamlitPatcher._wrap)

---

[source](https://github.com/ddobrinskiy/streamlit-jupyter/blob/master/streamlit_jupyter/core.py#L62){target="_blank" style="float:right; font-size:smaller"}

### StreamlitPatcher._wrap

>      StreamlitPatcher._wrap (cls, method_name:str, wrapper:Callable,
>                              verbose:bool=False)

make a streamlit method jupyter friendly

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| cls |  |  |  |
| method_name | str |  | which method to jupyterify |
| wrapper | typing.Callable |  | wrapper function to use |
| verbose | bool | False |  |
| **Returns** | **None** |  |  |

## Modifying streamlit

The way we will modify streamlit methods is by putting them through a decorator. This decorator will check if we are in a jupyter notebook, and if so, it will take the input and display it in the notebook. 

Else it will use the original streamlit method.

### st.write

In [ ]:
# | exporti


def _display(arg: tp.Any) -> None:
    if isinstance(arg, str):
        IPython.display.display(IPython.display.Markdown(arg))
    else:
        IPython.display.display(arg)


def _st_write(func_to_decorate):
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        for arg in args:
            _display(arg)

    return wrapper

In [ ]:
sp._wrap("write", _st_write)

with capture_output() as cap:
    st.write("hello")
    got = cap._outputs[0]["data"]

expected = {
    "text/plain": "<IPython.core.display.Markdown object>",
    "text/markdown": "hello",
}
assert got == expected, "check that the output is correct"

st.write("hello")

hello

In [ ]:
st.write("This is **bold** text in markdown")

This is **bold** text in markdown

In [ ]:
try:
    df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
    st.write(df)
except ImportError:
    logger.warning("Pandas not installed, skipping test")

,a,b
0,1,4
1,2,5
2,3,6


In [ ]:
assert sp.registered_methods == {"write"}, "check that the method is registered"

### patching headings

- `st.title`
- `st.header`
- `st.subheader`

In [ ]:
# | exporti
def _st_heading(func_to_decorate: tp.Callable, tag: str) -> tp.Callable:
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            body = args[0]
        elif len(args) == 2:
            body, anchor = args
        elif len(args) > 2:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )
        elif len(args) == 0:
            if "body" not in kwargs:
                raise ValueError(
                    f"Missing required argument: body, {func_to_decorate.__name__} requires a body"
                )
            body = kwargs["body"]

        if isinstance(body, str):
            _display(f"{tag} {body}")
        else:
            raise TypeError(
                f"Unsupported type: {type(body)}, {func_to_decorate.__name__} only accepts strings"
            )

    return wrapper

In [ ]:
sp = StreamlitPatcher()
sp._wrap("title", functools.partial(_st_heading, tag="#"))
sp._wrap("header", functools.partial(_st_heading, tag="##"))
sp._wrap("subheader", functools.partial(_st_heading, tag="###"))

In [ ]:
with capture_output() as cap:
    st.title("foo")
    got = cap._outputs[0]["data"]["text/markdown"]

test_eq(got, "# foo")

In [ ]:
with capture_output() as cap:
    st.header("foo")
    got = cap._outputs[0]["data"]["text/markdown"]

test_eq(got, "## foo")

In [ ]:
with capture_output() as cap:
    st.subheader("foo")
    got = cap._outputs[0]["data"]["text/markdown"]

test_eq(got, "### foo")

In [ ]:
# these should fail

test_fail(lambda: st.title(df), contains="Unsupported type")
test_fail(lambda: st.header(df), contains="Unsupported type")
test_fail(lambda: st.subheader(df), contains="Unsupported type")
test_fail(lambda: st.subheader(1), contains="Unsupported type")

### patch some methods to simply display the input in jupyter 

In [ ]:
# | exporti
def _st_type_check(
    func_to_decorate: tp.Callable,
    allowed_types: tp.Union[tp.Type, tp.Collection[tp.Type]],
) -> tp.Callable:
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""
    allowed_types = listify(allowed_types)  # make sure it's a list

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            body = args[0]
        elif len(args) > 1:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )
        elif len(args) == 0:
            if kwargs:
                raise NotImplementedError(
                    f"kwargs not supported yet, 'streamlit_data_science.utils._wrap_st_type_check' only accepts positional arguments"
                )
            else:
                raise ValueError(f"at least one positional argument is required")

        if type(body) in allowed_types:
            _display(body)
        else:
            raise TypeError(
                f"Unsupported type: {type(body)}, {func_to_decorate.__name__} only accepts {allowed_types}"
            )

    return wrapper

In [ ]:
sp._wrap("markdown", functools.partial(_st_type_check, allowed_types=str))

test_fail(lambda: st.markdown(df), contains="Unsupported type")
st.markdown("This is **bold** text in markdown")

This is **bold** text in markdown

In [ ]:
sp._wrap("dataframe", functools.partial(_st_type_check, allowed_types=pd.DataFrame))
test_fail(lambda: st.dataframe("foo"), contains="Unsupported type")
st.dataframe(df)

,a,b
0,1,4
1,2,5
2,3,6


### st.code

In [ ]:
_display(
    """```python
print(1+1)
```
"""
)

```python
print(1+1)
```


In [ ]:
# | exporti
def _st_code(func_to_decorate):
    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            body = args[0]
            language = kwargs["language"] if "language" in kwargs else "python"
        elif len(args) == 2:
            body, language = args

        if isinstance(body, str):
            _display(f"```{language}\n{body}\n```")
        else:
            raise TypeError(
                f"Unsupported type: {type(body)}, {func_to_decorate.__name__} only accepts strings"
            )

    return wrapper

In [ ]:
sp._wrap("code", _st_code)

In [ ]:
st.code(
    """
def foo():
    print('hello')
"""
)

```python

def foo():
    print('hello')

```

In [ ]:
st.code("grep -r 'foo' .", language=None)

```None
grep -r 'foo' .
```

### st.cache

The `streamlitcache` method is used to cache the output of a function. This is useful for functions that take a long time to run, and we want to avoid running them every time we run the app.

If we are in a jupyter notebook, we can't use the `streamlitcache` method, so we will replace the `streamlitcache` method with a dummy method that does nothing.

In [ ]:
# | exporti
def _dummy_wrapper_noop(func_to_decorate):
    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        return noop  # castrate the function to do nothing

    return wrapper

In [ ]:
sp._wrap("cache", _dummy_wrapper_noop)

In [ ]:
# verify that during patching we didn't change the name or docstring
assert st.cache.__name__ == "cache"
assert "@st.cache" in tp.cast(
    str, st.cache.__doc__
), "check that the docstring is correct"

In [ ]:
# test caching
@st.cache(suppress_st_warning=True)
def get_data():
    st.write("Getting data...")
    for i in tqdm(range(5)):
        time.sleep(0.1)
    return pd.DataFrame({"c": [7, 8, 9], "d": [10, 11, 12]})


df = get_data()
st.write(df)

Getting data...

  0%|          | 0/5 [00:00<?, ?it/s]

,c,d
0,7,10
1,8,11
2,9,12


In [ ]:
# test that the cache in jupyter does not affect get_data

df = get_data()
with capture_output() as cap:
    st.write(df)
    got = cap._outputs[0]["data"]

expected = {
    "text/plain": "   c   d\n0  7  10\n1  8  11\n2  9  12",
    "text/html": '<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>c</th>\n      <th>d</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>7</td>\n      <td>10</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>8</td>\n      <td>11</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>9</td>\n      <td>12</td>\n    </tr>\n  </tbody>\n</table>\n</div>',
}

assert got == expected, "check that the output is correct"

Getting data...

  0%|          | 0/5 [00:00<?, ?it/s]

### st.expander

Note that this will be an exception from the usual wrapper logic.

Since `st.expander` is used as a context manager, we replace it with a dummy class that displays the input in jupyter.

In [ ]:
# | exporti


class _DummyExpander:
    __doc__ = st.expander.__doc__

    def __init__(self, label: str, expanded: bool = False):
        self.label = label
        self.expanded = expanded

    def __enter__(self):
        _display(f">**expander starts**: {self.label}")

    def __exit__(self, *args):
        _display(f">**expander ends**")


def _st_expander(cls_to_replace: st.expander):
    return _DummyExpander

In [ ]:
sp._wrap("expander", _st_expander)

In [ ]:
st.expander.__doc__

'Insert a multi-element container that can be expanded/collapsed.\n\n        Inserts a container into your app that can be used to hold multiple elements\n        and can be expanded or collapsed by the user. When collapsed, all that is\n        visible is the provided label.\n\n        To add elements to the returned container, you can use "with" notation\n        (preferred) or just call methods directly on the returned object. See\n        examples below.\n\n        .. warning::\n            Currently, you may not put expanders inside another expander.\n\n        Parameters\n        ----------\n        label : str\n            A string to use as the header for the expander. The label can optionally\n            contain Markdown and supports the following elements: Bold, Italics,\n            Strikethroughs, Inline Code, Emojis, and Links.\n        expanded : bool\n            If True, initializes the expander in "expanded" state. Defaults to\n            False (collapsed).\n\n      

In [ ]:
with st.expander("Expand me!", expanded=False):
    st.markdown(
        """
The **#30DaysOfStreamlit** is a coding challenge designed to help you get started in building Streamlit apps.

Particularly, you'll be able to:
- Set up a coding environment for building Streamlit apps
- Build your first Streamlit app
- Learn about all the awesome input/output widgets to use for your Streamlit app
    """
    )

    st.write("**More text, we can expand as many streamlit elements as we want**")

>**expander starts**: Expand me!


The **#30DaysOfStreamlit** is a coding challenge designed to help you get started in building Streamlit apps.

Particularly, you'll be able to:
- Set up a coding environment for building Streamlit apps
- Build your first Streamlit app
- Learn about all the awesome input/output widgets to use for your Streamlit app
    

**More text, we can expand as many streamlit elements as we want**

>**expander ends**

### st.text_input

In [ ]:
# | exporti


def _st_text_input(func_to_decorate):
    """Decorator to display date input in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            description = args[0]
            if "value" in kwargs:
                value = kwargs["value"]
            else:
                value = None

        elif len(args) == 2:
            description, value = args

        text = widgets.Textarea(
            description=description,
            value=value,
            disabled=False,
            placeholder="Type something",
        )

        display(text)
        return text.value

    return wrapper

In [ ]:
sp._wrap("text_input", _st_text_input)
sp._wrap("text_area", _st_text_input)

In [ ]:
text = st.text_input("String:", "default text")
text

Textarea(value='default text', description='String:', placeholder='Type something')

'default text'

In [ ]:
text = st.text_area("Input:", "foo bar")
text

Textarea(value='foo bar', description='Input:', placeholder='Type something')

'foo bar'

### st.date_input


In [ ]:
# | exporti


def _st_date_input(func_to_decorate):
    """Decorator to display date input in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            description = args[0]
            if "value" in kwargs:
                value = pd.to_datetime(kwargs["value"]).date()
            else:
                value = datetime.now()

        elif len(args) == 2:
            description = args[0]
            value = pd.to_datetime(args[1])

        date = widgets.DatePicker(
            description=description,
            value=value,
            disabled=False,
        )

        display(date)
        return date.value

    return wrapper

In [ ]:
sp._wrap("date_input", _st_date_input)

⚠️ Note the following limitation: when using this in jupyter, changing the date on your widget will not affect the date variable.

Streamlit behavior will remain unchanged though

In [ ]:
date = st.date_input("Pick a date", value="2022-12-13")

DatePicker(value=datetime.date(2022, 12, 13), description='Pick a date')

In [ ]:
assert date == datetime(2022, 12, 13).date()

### st.checkbox

In [ ]:
# | exporti


def _st_checkbox(func_to_decorate):
    """Decorator to display checkbox in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            description = args[0]
            if "value" in kwargs:
                value = kwargs["value"]
            else:
                value = True

        elif len(args) == 2:
            description, value = args

        w = widgets.Checkbox(
            value=value, description=description, disabled=False, indent=False
        )

        display(w)
        return w.value

    return wrapper

In [ ]:
sp._wrap("checkbox", _st_checkbox)

In [ ]:
show_code = st.checkbox("Show code")
assert show_code

Checkbox(value=True, description='Show code', indent=False)

In [ ]:
show_code = st.checkbox("Show code", value=False)
assert not show_code

Checkbox(value=False, description='Show code', indent=False)

# _st_radio and _st_selectbox

In [ ]:
# | exporti


def _st_single_choice(func_to_decorate, jupyter_widget: widgets.Widget):

    """Decorator to display single choice widget in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) < 1:
            raise ValueError("You must provide at least 1 argument")
        if len(args) == 1:
            label = args[0]
            options = kwargs["options"]
            index = kwargs["index"] if "index" in kwargs else 0
        elif len(args) == 2:
            label, options = args
            index = kwargs["index"] if "index" in kwargs else 0
        elif len(args) == 3:
            label, options, index = args

        w = jupyter_widget(
            options=options,
            description=label,
            index=index,
        )

        display(w)
        return w.value

    return wrapper

In [ ]:
sp._wrap(
    "radio", functools.partial(_st_single_choice, jupyter_widget=widgets.RadioButtons)
)
sp._wrap(
    "selectbox", functools.partial(_st_single_choice, jupyter_widget=widgets.Dropdown)
)

In [ ]:
st.radio("Pick", options=["foo", "bar"], index=1, key="radio")

RadioButtons(description='Pick', index=1, options=('foo', 'bar'), value='bar')

'bar'

In [ ]:
st.selectbox("Choose", options=["foo", "bar"])

Dropdown(description='Choose', options=('foo', 'bar'), value='foo')

'foo'

### st.multiselect

In [ ]:
# | exporti


def _st_multiselect(func_to_decorate):
    """Decorator to display multiple choice widget in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) < 1:
            raise ValueError("You must provide at least 1 argument")
        if len(args) == 1:
            label = args[0]
            options = kwargs.get("options")
        elif len(args) == 2:
            label, options = args
            index = kwargs["index"] if "index" in kwargs else 0
        else:
            raise ValueError("Too many positional arguments, provide at most 2")

        w = widgets.SelectMultiple(
            options=options,
            description=label,
            value=kwargs.get("default", []),
        )

        display(w)
        return w.value

    return wrapper

In [ ]:
sp._wrap("multiselect", _st_multiselect)

In [ ]:
st.multiselect("Multiselect: ", options=["python", "golang", "julia", "rust"])

SelectMultiple(description='Multiselect: ', options=('python', 'golang', 'julia', 'rust'), value=())

()

In [ ]:
st.multiselect(
    "Multiselect with defaults: ",
    options=["nbdev", "streamlit", "jupyter", "fastcore"],
    default=["jupyter", "streamlit"],
)

SelectMultiple(description='Multiselect with defaults: ', index=(2, 1), options=('nbdev', 'streamlit', 'jupyte…

('jupyter', 'streamlit')

### StreamlitPatcher.MAPPING

Mapping is a dictionary that maps the streamlit method to the method we want to use instead.

This is used when StreamlitPatcher.jupyter() is called.

In [ ]:
# | exporti
@patch_to(StreamlitPatcher, as_prop=True)
def MAPPING(cls) -> tp.Dict[str, tp.Callable]:
    """mapping of streamlit methods to their jupyter friendly versions"""
    return {
        "write": _st_write,
        "title": functools.partial(_st_heading, tag="#"),
        "header": functools.partial(_st_heading, tag="##"),
        "subheader": functools.partial(_st_heading, tag="###"),
        "markdown": functools.partial(_st_type_check, allowed_types=str),
        "dataframe": functools.partial(_st_type_check, allowed_types=pd.DataFrame),
        "date_input": _st_date_input,
        "cache": _dummy_wrapper_noop,
        "expander": _st_expander,
        "text_input": _st_text_input,
        "text_area": _st_text_input,
        "code": _st_code,
        "checkbox": _st_checkbox,
        "radio": functools.partial(
            _st_single_choice, jupyter_widget=widgets.RadioButtons
        ),
        "selectbox": functools.partial(
            _st_single_choice, jupyter_widget=widgets.Dropdown
        ),
        "multiselect": _st_multiselect,
    }

In [ ]:
sp = StreamlitPatcher()

In [ ]:
assert not sp.registered_methods, "registered methods should be empty at this point"

In [ ]:
# | hide
sp.MAPPING

{'write': <function __main__._st_write(func_to_decorate)>,
 'title': functools.partial(<function _st_heading>, tag='#'),
 'header': functools.partial(<function _st_heading>, tag='##'),
 'subheader': functools.partial(<function _st_heading>, tag='###'),
 'markdown': functools.partial(<function _st_type_check>, allowed_types=<class 'str'>),
 'dataframe': functools.partial(<function _st_type_check>, allowed_types=<class 'pandas.core.frame.DataFrame'>),
 'date_input': <function __main__._st_date_input(func_to_decorate)>,
 'cache': <function __main__._dummy_wrapper_noop(func_to_decorate)>,
 'expander': <function __main__._st_expander(cls_to_replace: <bound method LayoutsMixin.expander of DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)>)>,
 'text_input': <function __main__._st_text_input(func_to_decorate)>,
 'text_area': <function __main__._st_text_input(func_to_decorate)>,
 'code': <function __main__._st_code(func_to_decorate)>,
 'ch

In [ ]:
show_doc(StreamlitPatcher.jupyter)

---

[source](https://github.com/ddobrinskiy/streamlit-jupyter/blob/master/streamlit_jupyter/core.py#L47){target="_blank" style="float:right; font-size:smaller"}

### StreamlitPatcher.jupyter

>      StreamlitPatcher.jupyter (verbose:bool=False)

patches streamlit methods to display content in jupyter notebooks

In [ ]:
sp.jupyter()
sp.registered_methods

{'cache',
 'checkbox',
 'code',
 'dataframe',
 'date_input',
 'expander',
 'header',
 'markdown',
 'multiselect',
 'radio',
 'selectbox',
 'subheader',
 'text_area',
 'text_input',
 'title',
 'write'}

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()